In [1]:
from pyscf import scf,gto
from functools import reduce
import numpy as np
import inspect
from FcMole import FcM
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian.rhf import * # without those two mf.Gradients() and mf.Hessian() don't work
from pyscf.hessian import rhf as hrhf
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
mol=gto.M(atom="H 0 0 0; Li 0 0 1.8",unit="Bohr",basis="STO-3G")
mf=scf.RHF(mol)
mf.scf(dm0=mf.init_guess_by_atom())
P=mf.make_rdm1()
hcore=mf.get_hcore()
g=mf.Gradients()
h=mf.Hessian()
h.kernel()
pass

converged SCF energy = -7.74106569060784


/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/lib/chkfile.py:71: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return val.value


In [3]:
mol1=gto.M(atom="H 0 0 0; Li 0 0 1.81",unit="Bohr",basis="STO-3G")
mf1=scf.RHF(mol1)
mf1.scf()
hcore1=mf1.get_hcore()
P1=mf1.make_rdm1()
h1=mf1.Hessian()
h1.kernel()
pass

converged SCF energy = -7.74424391363581


In [4]:
mo_energy = mf.mo_energy
mo_occ = mf.mo_occ
mo_coeff = mf.mo_coeff
C=mo_coeff
atmlst = range(mol.natm)
mocc = mo_coeff[:,mo_occ>0]

In [5]:
h1ao = h.make_h1(mo_coeff,mo_occ)
np.allclose(h1ao[1][2],-h1ao[0][2])

True

In [6]:
print(inspect.getsource(h.make_h1))   ##### takes into account also 

def make_h1(hessobj, mo_coeff, mo_occ, chkfile=None, atmlst=None, verbose=None):
    time0 = t1 = (time.clock(), time.time())
    mol = hessobj.mol
    if atmlst is None:
        atmlst = range(mol.natm)

    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    dm0 = numpy.dot(mocc, mocc.T) * 2
    hcore_deriv = hessobj.base.nuc_grad_method().hcore_generator(mol)

    aoslices = mol.aoslice_by_atom()
    h1ao = [None] * mol.natm
    for i0, ia in enumerate(atmlst):
        shl0, shl1, p0, p1 = aoslices[ia]
        shls_slice = (shl0, shl1) + (0, mol.nbas)*3
        vj1, vj2, vk1, vk2 = _get_jk(mol, 'int2e_ip1', 3, 's2kl',
                                     ['ji->s2kl', -dm0[:,p0:p1],  # vj1
                                      'lk->s1ij', -dm0         ,  # vj2
                                      'li->s1kj', -dm0[:,p0:p1],  # vk1
                                      'jk->s1il', -dm0         ], # vk2
                                     shls_slice=shls_slice)
        vhf =

In [7]:
np.round(h.base.nuc_grad_method().hcore_generator(mol)(1)[2],decimals=3) #truly the derivative of h1 in ao basis

array([[ 0.763,  1.321,  0.419, -0.   , -0.   , -0.002],
       [ 1.321,  0.308,  0.067, -0.   , -0.   , -0.037],
       [ 0.419,  0.067,  0.049, -0.   , -0.   ,  0.011],
       [-0.   , -0.   , -0.   ,  0.059, -0.   , -0.   ],
       [-0.   , -0.   , -0.   , -0.   ,  0.059, -0.   ],
       [-0.002, -0.037,  0.011, -0.   , -0.   ,  0.037]])

In [8]:
np.round((hcore1-hcore)*100,decimals=3)

array([[ 0.761,  1.315,  0.419,  0.   ,  0.   , -0.004],
       [ 1.315,  0.306,  0.067,  0.   ,  0.   , -0.037],
       [ 0.419,  0.067,  0.05 ,  0.   ,  0.   ,  0.011],
       [ 0.   ,  0.   ,  0.   ,  0.059,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.059,  0.   ],
       [-0.004, -0.037,  0.011,  0.   ,  0.   ,  0.037]])

In [10]:
 print(inspect.getsource(h.hess_elec)) 

def hess_elec(hessobj, mo_energy=None, mo_coeff=None, mo_occ=None,
              mo1=None, mo_e1=None, h1ao=None,
              atmlst=None, max_memory=4000, verbose=None):
    log = logger.new_logger(hessobj, verbose)
    time0 = t1 = (time.clock(), time.time())

    mol = hessobj.mol
    mf = hessobj.base
    if mo_energy is None: mo_energy = mf.mo_energy
    if mo_occ is None:    mo_occ = mf.mo_occ
    if mo_coeff is None:  mo_coeff = mf.mo_coeff
    if atmlst is None: atmlst = range(mol.natm)

    de2 = hessobj.partial_hess_elec(mo_energy, mo_coeff, mo_occ, atmlst,
                                    max_memory, log)

    if h1ao is None:
        h1ao = hessobj.make_h1(mo_coeff, mo_occ, hessobj.chkfile, atmlst, log)
        t1 = log.timer_debug1('making H1', *time0)
    if mo1 is None or mo_e1 is None:
        mo1, mo_e1 = hessobj.solve_mo1(mo_energy, mo_coeff, mo_occ, h1ao,
                                       None, atmlst, max_memory, log)
        t1 = log.timer_debug1('solving

In [13]:
print(inspect.getsource(g.grad_elec))

def grad_elec(mf_grad, mo_energy=None, mo_coeff=None, mo_occ=None, atmlst=None):
    '''
    Electronic part of RHF/RKS gradients

    Args:
        mf_grad : grad.rhf.Gradients or grad.rks.Gradients object
    '''
    mf = mf_grad.base
    mol = mf_grad.mol
    if mo_energy is None: mo_energy = mf.mo_energy
    if mo_occ is None:    mo_occ = mf.mo_occ
    if mo_coeff is None:  mo_coeff = mf.mo_coeff
    log = logger.Logger(mf_grad.stdout, mf_grad.verbose)

    hcore_deriv = mf_grad.hcore_generator(mol)
    s1 = mf_grad.get_ovlp(mol)
    dm0 = mf.make_rdm1(mo_coeff, mo_occ)

    t0 = (time.clock(), time.time())
    log.debug('Computing Gradients of NR-HF Coulomb repulsion')
    vhf = mf_grad.get_veff(mol, dm0)
    log.timer('gradients of 2e part', *t0)

    dme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)

    if atmlst is None:
        atmlst = range(mol.natm)
    aoslices = mol.aoslice_by_atom()
    de = numpy.zeros((len(atmlst),3))
    for k, ia in enumerate(atmlst):
        p